Rasmika Billa

Installing and Importing the required Libraries

In [150]:
pip install wordsegment

In [151]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.base import TransformerMixin

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

import re
import nltk
from nltk.util import ngrams  
import collections
from nltk.corpus import stopwords
from nltk import word_tokenize
from wordsegment import segment, load
from nltk.tokenize import TweetTokenizer
nltk.download("stopwords")
load()
replace_by_space = re.compile('[/(){}\[\]\|@,;]')
replace_symbol = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

from keras.models import Sequential
from keras.layers import Dense

from scipy.stats import spearmanr
from scipy.stats import pearsonr

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Importing Datasets

In [152]:
train = pd. read_csv("joy-ratings-0to1.train.txt", sep="\t", names=['id', 'tweet', 'emotion', 'score'])
val = pd. read_csv("joy-ratings-0to1.dev.target.txt", sep="\t", names=['id', 'tweet', 'emotion', 'score'])
val_gold = pd. read_csv("joy-ratings-0to1.dev.gold.txt", sep="\t", names=['id', 'tweet', 'emotion', 'score'])
test = pd. read_csv("joy-ratings-0to1.test.target.txt", sep="\t", names=['id', 'tweet', 'emotion', 'score'])
test_gold = pd. read_csv("joy-ratings-0to1.test.gold.txt", sep="\t", names=['id', 'tweet', 'emotion', 'score'])

In [153]:
pip install emot

Preprocessing the given data

In [155]:
tknzr = TweetTokenizer(reduce_len=True, preserve_case=False, strip_handles=False)
from emot.emo_unicode import UNICODE_EMOJI
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

def text_preprocess(text):
    FLAGS = re.MULTILINE | re.DOTALL
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")
    text = re_sub(r"#\S+", lambda hashtag: " ".join(segment(hashtag.group()[1:]))) 

    tokens = tknzr.tokenize(text.lower())
    return " ".join(tokens)



Embedding the metrics given for the task

In [156]:
def metric(y_pred, y, title):
    p1 = pearsonr(y_pred, y)[0]
    s1 = spearmanr(y_pred, y)[0]
    ind = np.where(y >= 0.5)
    ydt = np.take(y_pred, ind).reshape(-1)
    ydpt = np.take(y.to_numpy(), ind).reshape(-1)
    p2 = pearsonr(ydt, ydpt)[0]
    s2 = spearmanr(ydt, ydpt)[0]
    print(title)
    res = {}
    res['Pearsonr'] = p1
    res['Spearmanr'] = s1
    res['Pearsonr >= 0.5'] = p2
    res['Spearmanr >= 0.5'] = s2
    result = pd.DataFrame(res.items(), columns=['metrics', 'Value'], index=None)
    return result

The ML algos used - Random Forest and Support Vector Machine

In [157]:
def ml_model(X_train, X_test, y_train, y_test, val_X, val_y):
    prediction_accuracy = {}
    
    # RF
    _RF = RandomForestRegressor(n_estimators = 1000, oob_score=True)
    _RF.fit(X_train, y_train)
    _RF_prediction = _RF.predict(X_test)
    val_RF_prediction = _RF.predict(val_X)

    print(metric(_RF_prediction, y_test, "Train-set "))
    print(metric(val_RF_prediction, val_y, "Test-set "))
    print('\n ')
    
    # SVM
    _SVC = SVR( kernel = 'rbf')
    _SVC.fit(X_train, y_train)
    _SVC_prediction = _SVC.predict(X_test)
    val_SVC_prediction = _SVC.predict(val_X)
    
    print(metric(_SVC_prediction, y_test, "Train-set "))
    print(metric(val_SVC_prediction, val_y, "Test-set "))
    print('\n ')
    
    #accuracy DataFram
    prediction_accuracy_df = pd.DataFrame(prediction_accuracy.items(), columns=['Model', 'Accuracy'], index=None)
    return prediction_accuracy_df

DL Vectorization 

In [158]:
def Vectorization(X, y, val_X, val_y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
    tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=1000, min_df=20, stop_words= STOPWORDS )
    X_train = tfidf.fit_transform(X_train).toarray()
    X_test = tfidf.transform(X_test).toarray()
    val_X = tfidf.transform(val_X).toarray()
    return X_train, X_test, y_train, y_test, val_X, val_y

In [159]:
def train_ml(train, val, test, prePro=False):
  train_val = pd.concat([train, val], ignore_index=True)
  if prePro:
    train['tweet'] = train['tweet'].apply(convert_emojis)
    val['tweet'] = val['tweet'].apply(convert_emojis)
    test['tweet'] = test['tweet'].apply(convert_emojis)
    train_val['tweet'] = train_val['tweet'].apply(convert_emojis)

    train['tweet'] = train['tweet'].apply(text_preprocess)
    val['tweet'] = val['tweet'].apply(text_preprocess)
    test['tweet'] = test['tweet'].apply(text_preprocess)
    train_val['tweet'] = train_val['tweet'].apply(text_preprocess)
  

  # Train
  print("Train")
  X_train, X_test, y_train, y_test,val_X, val_y = Vectorization(train.tweet ,train.score , val_gold.tweet , val_gold.score )
  p_df = ml_model(X_train, X_test, y_train, y_test, val_X, val_y)
  # Test
  print("Train+Test")
  X_train, X_test, y_train, y_test,val_X, val_y = Vectorization(train_val.tweet ,train_val.score , test_gold.tweet , test_gold.score)
  p_df = ml_model(X_train, X_test, y_train, y_test, val_X, val_y)

Values before pre-processing

In [160]:
train_ml(train, val_gold, test_gold, prePro=False)

Train
Train-set 
            metrics     Value
0          Pearsonr  0.499893
1         Spearmanr  0.452352
2   Pearsonr >= 0.5  0.410774
3  Spearmanr >= 0.5  0.391833
Test-set 
            metrics     Value
0          Pearsonr  0.292431
1         Spearmanr  0.335752
2   Pearsonr >= 0.5  0.257776
3  Spearmanr >= 0.5  0.304652

 
Train-set 
            metrics     Value
0          Pearsonr  0.473967
1         Spearmanr  0.437280
2   Pearsonr >= 0.5  0.385501
3  Spearmanr >= 0.5  0.404218
Test-set 
            metrics     Value
0          Pearsonr  0.256570
1         Spearmanr  0.293664
2   Pearsonr >= 0.5  0.145374
3  Spearmanr >= 0.5  0.199536

 
Train+Test
Train-set 
            metrics     Value
0          Pearsonr  0.429250
1         Spearmanr  0.375891
2   Pearsonr >= 0.5  0.512225
3  Spearmanr >= 0.5  0.462843
Test-set 
            metrics     Value
0          Pearsonr  0.312361
1         Spearmanr  0.309019
2   Pearsonr >= 0.5  0.153941
3  Spearmanr >= 0.5  0.157264

 
Train-set 


Values after pre-processing

In [161]:
train_ml(train, val_gold, test_gold, prePro=True)

Train
Train-set 
            metrics     Value
0          Pearsonr  0.481537
1         Spearmanr  0.447167
2   Pearsonr >= 0.5  0.323882
3  Spearmanr >= 0.5  0.321267
Test-set 
            metrics     Value
0          Pearsonr  0.292624
1         Spearmanr  0.331025
2   Pearsonr >= 0.5  0.217924
3  Spearmanr >= 0.5  0.189735

 
Train-set 
            metrics     Value
0          Pearsonr  0.494236
1         Spearmanr  0.471457
2   Pearsonr >= 0.5  0.348926
3  Spearmanr >= 0.5  0.364503
Test-set 
            metrics     Value
0          Pearsonr  0.218412
1         Spearmanr  0.249152
2   Pearsonr >= 0.5  0.140314
3  Spearmanr >= 0.5  0.134655

 
Train+Test
Train-set 
            metrics     Value
0          Pearsonr  0.464138
1         Spearmanr  0.436171
2   Pearsonr >= 0.5  0.507386
3  Spearmanr >= 0.5  0.492674
Test-set 
            metrics     Value
0          Pearsonr  0.258063
1         Spearmanr  0.256257
2   Pearsonr >= 0.5  0.158982
3  Spearmanr >= 0.5  0.157210

 
Train-set 


DL Neural network model

In [162]:
def NN_model(X_train, X_test, y_train, y_test, val_X, val_y):

  model = Sequential()
  model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(1, kernel_initializer='normal',activation='linear'))
  model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

  model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split = 0.2)
  pre = model.predict(X_test)
  p = []
  for i in pre:
    p.append(i[0])
  print(metric(p, y_test, "Train-set "))
  pre_val = model.predict(val_X)
  p_val = []
  for i in pre_val:
    p_val.append(i[0])
  print(metric(p_val, val_y, "Test-set "))
  
  return None

In [163]:
def train_NN(train, val, test, prePro=False):
  train_val = pd.concat([train, val], ignore_index=True)
  if prePro:
    train['tweet'] = train['tweet'].apply(convert_emojis)
    val['tweet'] = val['tweet'].apply(convert_emojis)
    test['tweet'] = test['tweet'].apply(convert_emojis)
    train_val['tweet'] = train_val['tweet'].apply(convert_emojis)

    train['tweet'] = train['tweet'].apply(text_preprocess)
    val['tweet'] = val['tweet'].apply(text_preprocess)
    test['tweet'] = test['tweet'].apply(text_preprocess)
    train_val['tweet'] = train_val['tweet'].apply(text_preprocess)

  # Train
  print("Train")
  X_train, X_test, y_train, y_test,val_X, val_y = Vectorization(train.tweet ,train.score , val_gold.tweet , val_gold.score )
  p_df = NN_model(X_train, X_test, y_train, y_test, val_X, val_y)
  # Test
  print("Train+Test")
  X_train, X_test, y_train, y_test,val_X, val_y = Vectorization(train_val.tweet ,train_val.score , test_gold.tweet , test_gold.score)
  p_df = NN_model(X_train, X_test, y_train, y_test, val_X, val_y)

Before pre-processing

In [164]:
train_NN(train, val_gold, test_gold, prePro=False)

Train
Epoch 1/100
17/17 [==============================] - 1s 16ms/step - loss: 0.2742 - mean_absolute_error: 0.2742 - val_loss: 0.1616 - val_mean_absolute_error: 0.1616
Epoch 2/100
17/17 [==============================] - 0s 6ms/step - loss: 0.1785 - mean_absolute_error: 0.1785 - val_loss: 0.1556 - val_mean_absolute_error: 0.1556
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1640 - mean_absolute_error: 0.1640 - val_loss: 0.1660 - val_mean_absolute_error: 0.1660
Epoch 4/100
17/17 [==============================] - 0s 6ms/step - loss: 0.1560 - mean_absolute_error: 0.1560 - val_loss: 0.1560 - val_mean_absolute_error: 0.1560
Epoch 5/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1526 - mean_absolute_error: 0.1526 - val_loss: 0.1608 - val_mean_absolute_error: 0.1608
Epoch 6/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1485 - mean_absolute_error: 0.1485 - val_loss: 0.1504 - val_mean_absolute_error: 0.1504
Epoch 7/100
17/

After pre-processing

In [165]:
train_NN(train, val_gold, test_gold, prePro=True)

Train
Epoch 1/100
17/17 [==============================] - 1s 15ms/step - loss: 0.2966 - mean_absolute_error: 0.2966 - val_loss: 0.2061 - val_mean_absolute_error: 0.2061
Epoch 2/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1681 - mean_absolute_error: 0.1681 - val_loss: 0.1600 - val_mean_absolute_error: 0.1600
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1541 - mean_absolute_error: 0.1541 - val_loss: 0.1505 - val_mean_absolute_error: 0.1505
Epoch 4/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1577 - mean_absolute_error: 0.1577 - val_loss: 0.1540 - val_mean_absolute_error: 0.1540
Epoch 5/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1495 - mean_absolute_error: 0.1495 - val_loss: 0.1514 - val_mean_absolute_error: 0.1514
Epoch 6/100
17/17 [==============================] - 0s 6ms/step - loss: 0.1502 - mean_absolute_error: 0.1502 - val_loss: 0.1511 - val_mean_absolute_error: 0.1511
Epoch 7/100
17/